# Install pysyft For CoLab

In [1]:
# uncomment for colab
# !pip install syft

     |████████████████████████████████| 256kB 2.8MB/s 
     |████████████████████████████████| 204kB 42.3MB/s 
     |████████████████████████████████| 256kB 39.7MB/s 
     |████████████████████████████████| 1.4MB 40.9MB/s 
     |████████████████████████████████| 460kB 41.6MB/s 
     |████████████████████████████████| 389kB 41.4MB/s 
     |████████████████████████████████| 81kB 23.3MB/s 
     |████████████████████████████████| 266kB 40.3MB/s 
     |████████████████████████████████| 51kB 18.2MB/s 
     |████████████████████████████████| 122kB 42.8MB/s 
  Created wheel for zstd: filename=zstd-1.4.1.0-cp36-cp36m-linux_x86_64.whl size=1067085 sha256=d92af2f6da57f73161cc9aa98db938bdaa2469a18f914fd59032a73bff4d4abf
  Stored in directory: /root/.cache/pip/wheels/66/3f/ee/ac08c81af7c1b24a80c746df669ea3cb37542d27877d66ccf4
  Created wheel for pyyaml: filename=PyYAML-5.1.2-cp36-cp36m-linux_x86_64.whl size=44105 sha256=1d79db30d8c17e6e9788a33205807eef37648e860aad5528f484048a80b038cc
  Stored in di

# Section: Securing Federated Learning

- Lesson 1: Trusted Aggregator
- Lesson 2: Intro to Additive Secret Sharing
- Lesson 3: Intro to Fixed Precision Encoding
- Lesson 4: Secret Sharing + Fixed Precision in PySyft
- Final Project: Federated Learning wtih Encrypted Gradient Aggregation

# Lesson: Federated Learning with a Trusted Aggregator

In the last section, we learned how to train a model on a distributed dataset using Federated Learning. In particular, the last project aggregated gradients directly from one data owner to another. 

However, while in some cases it could be ideal to do this, what would be even better is to be able to choose a neutral third party to perform the aggregation.

As it turns out, we can use the same tools we used previously to accomplish this.

# Project: Federated Learning with a Trusted Aggregator

In [0]:
# try this project here!

In [1]:
import syft as sy
import torch as th
hook = sy.TorchHook(th)
from torch import nn, optim

W0808 17:57:38.079879 4712072640 secure_random.py:22] Falling back to insecure randomness since the required custom op could not be found for the installed version of TensorFlow (1.14.0). Fix this by compiling custom ops.
W0808 17:57:38.498445 4712072640 deprecation_wrapper.py:119] From /Users/lek/miniconda3/envs/pysyft/lib/python3.6/site-packages/tf_encrypted/session.py:28: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.



In [2]:
# create a couple worker
bob = sy.VirtualWorker(hook, id='bob')
alice = sy.VirtualWorker(hook, id='alice')
secure_worker = sy.VirtualWorker(hook, id='secure_worker')

In [3]:
bob.add_workers([alice, secure_worker])
alice.add_workers([bob, secure_worker])
secure_worker.add_workers([alice, bob])

W0808 17:57:53.870684 4712072640 base.py:628] Worker alice already exists. Replacing old worker which could cause                     unexpected behavior
W0808 17:57:53.872111 4712072640 base.py:628] Worker secure_worker already exists. Replacing old worker which could cause                     unexpected behavior
W0808 17:57:53.873852 4712072640 base.py:628] Worker bob already exists. Replacing old worker which could cause                     unexpected behavior
W0808 17:57:53.874777 4712072640 base.py:628] Worker secure_worker already exists. Replacing old worker which could cause                     unexpected behavior
W0808 17:57:53.875526 4712072640 base.py:628] Worker alice already exists. Replacing old worker which could cause                     unexpected behavior
W0808 17:57:53.876311 4712072640 base.py:628] Worker bob already exists. Replacing old worker which could cause                     unexpected behavior


<VirtualWorker id:secure_worker #objects:0>

In [4]:
# A toy Dataset
data = th.tensor([[0,0], [0,1], [1,0], [1,1.]], requires_grad=True)
target = th.tensor([[0], [0], [1], [1.]], requires_grad=True)

In [5]:
# get pointers to training data on each worker by
# sending some training data to bob and alice
bobs_data = data[0:2].send(bob)
bobs_target = target[0:2].send(bob)

In [6]:
alices_data = data[2:].send(alice)
alices_target = target[2:].send(alice)

In [7]:
# Initialize A Toy Model
model = nn.Linear(2,1)

In [8]:
bobs_model = model.copy().send(bob)

In [9]:
alices_model = model.copy().send(alice)

In [10]:
bobs_opt = optim.SGD(params=bobs_model.parameters(), lr=0.1)
alices_opt = optim.SGD(params=alices_model.parameters(), lr=0.1)

In [11]:
bobs_opt.zero_grad()
bobs_pred = bobs_model(bobs_data)
bobs_loss = ((bobs_pred - bobs_target) ** 2).sum()
bobs_loss.backward()

(Wrapper)>[PointerTensor | me:66708618334 -> bob:42499187294]

In [12]:
bobs_opt.step()

In [13]:
bobs_loss = bobs_loss.get().data
bobs_loss

tensor(0.9939)

In [14]:
alices_opt.zero_grad()
alices_pred = alices_model(alices_data)
alices_loss = ((alices_pred - alices_target) ** 2).sum()
alices_loss.backward()

alices_opt.step()
alices_loss = alices_loss.get().data
alices_loss

tensor(2.4496)

In [15]:
for i in range(10):
    bobs_opt.zero_grad()
    bobs_pred = bobs_model(bobs_data)
    bobs_loss = ((bobs_pred - bobs_target) ** 2).sum()
    bobs_loss.backward()
    
    bobs_opt.step()
    bobs_loss = bobs_loss.get().data
    bobs_loss
    
    alices_opt.zero_grad()
    alices_pred = alices_model(alices_data)
    alices_loss = ((alices_pred - alices_target) ** 2).sum()
    alices_loss.backward()

    alices_opt.step()
    alices_loss = alices_loss.get().data
    alices_loss

In [16]:
bobs_model.move(secure_worker)
alices_model.move(secure_worker)

In [17]:
secure_worker._objects

{88322742208: Parameter containing:
 tensor([[ 0.5718, -0.1223]], requires_grad=True),
 7706265944: Parameter containing:
 tensor([0.0754], requires_grad=True),
 45656704075: Parameter containing:
 tensor([[ 0.9864, -0.1247]], requires_grad=True),
 70618331248: Parameter containing:
 tensor([0.0836], requires_grad=True)}

In [18]:
alices_model

Linear(in_features=2, out_features=1, bias=True)

In [19]:
alices_model.weight

Parameter containing:
Parameter>[PointerTensor | me:45656704075 -> secure_worker:45656704075]

In [20]:
alices_model.weight.data

(Wrapper)>[PointerTensor | me:45656704075 -> secure_worker:45656704075]::data

In [21]:
model.weight.data = (((alices_model.weight.data + bobs_model.weight.data) / 2).get())

In [22]:
model.weight.data

tensor([[ 0.7791, -0.1235]])

In [23]:
model.bias.data = (((alices_model.bias.data + bobs_model.bias.data) / 2).get())

In [24]:
model.bias.data

tensor([0.0795])

In [25]:
for round_iter in range(10):
    
    bobs_model = model.copy().send(bob)
    alices_model = model.copy().send(alice)
    
    bobs_opt = optim.SGD(params=bobs_model.parameters(), lr=0.1)
    alices_opt = optim.SGD(params=alices_model.parameters(), lr=0.1)
    
    for i in range(10):
        bobs_opt.zero_grad()
        bobs_pred = bobs_model(bobs_data)
        bobs_loss = ((bobs_pred - bobs_target) ** 2).sum()
        bobs_loss.backward()

        bobs_opt.step()
        bobs_loss = bobs_loss.get().data
        bobs_loss

        alices_opt.zero_grad()
        alices_pred = alices_model(alices_data)
        alices_loss = ((alices_pred - alices_target) ** 2).sum()
        alices_loss.backward()

        alices_opt.step()
        alices_loss = alices_loss.get().data
        alices_loss
        
    bobs_model.move(secure_worker)
    alices_model.move(secure_worker)
    
    model.weight.data = (((alices_model.weight.data + bobs_model.weight.data) / 2).get())
    model.bias.data = (((alices_model.bias.data + bobs_model.bias.data) / 2).get())
    
    secure_worker.clear_objects()
    
    print('Bob:' + str(bobs_loss) + 'Alice:' + str(alices_loss))

Bob:tensor(0.0020)Alice:tensor(0.0005)
Bob:tensor(0.0007)Alice:tensor(2.5695e-05)
Bob:tensor(0.0003)Alice:tensor(3.1932e-07)
Bob:tensor(0.0002)Alice:tensor(6.4014e-06)
Bob:tensor(0.0001)Alice:tensor(9.5086e-06)
Bob:tensor(7.9822e-05)Alice:tensor(9.4158e-06)
Bob:tensor(5.7255e-05)Alice:tensor(8.0469e-06)
Bob:tensor(4.2017e-05)Alice:tensor(6.4443e-06)
Bob:tensor(3.1208e-05)Alice:tensor(5.0028e-06)
Bob:tensor(2.3326e-05)Alice:tensor(3.8247e-06)


# Lesson: Intro to Additive Secret Sharing

While being able to have a trusted third party to perform the aggregation is certainly nice, in an ideal setting we wouldn't have to trust anyone at all. This is where Cryptography can provide an interesting alterantive. 

Specifically, we're going to be looking at <font color=red>a simple protocol for Secure Multi-Party Computation called Additive Secret Sharing.</font> <font color=blue>This protocol will allow multiple parties (of size 3 or more) to aggregate their gradients without the use of a trusted 3rd party to perform the aggregation.</font><font color=red> In other words, we can add 3 numbers together from 3 different people without anyone ever learning the inputs of any other actors.</font>

Let's start by considering the number 5, which we'll put into a varible x

In [26]:
x = 5

Let's say we wanted to SHARE the ownership of this number between two people, Alice and Bob. We could split this number into two shares, 2, and 3, and give one to Alice and one to Bob

In [27]:
bob_x_share = 2
alice_x_share = 3

decrypted_x = bob_x_share + alice_x_share
decrypted_x

5

Note that neither Bob nor Alice know the value of x. They only know the value of their own SHARE of x. Thus, the true value of X is hidden (i.e., encrypted). 

The truly amazing thing, however, is that Alice and Bob can still compute using this value! They can perform arithmetic over the hidden value! Let's say Bob and Alice wanted to multiply this value by 2! If each of them multiplied their respective share by 2, then the hidden number between them is also multiplied! Check it out!

In [28]:
bob_x_share = 2 * 2
alice_x_share = 3 * 2

decrypted_x = bob_x_share + alice_x_share
decrypted_x

10

This even works for addition between two shared values!!

In [29]:
# encrypted "5"
bob_x_share = 2
alice_x_share = 3

# encrypted "7"
bob_y_share = 5
alice_y_share = 2

# encrypted 5 + 7
bob_z_share = bob_x_share + bob_y_share
alice_z_share = alice_x_share + alice_y_share

decrypted_z = bob_z_share + alice_z_share
decrypted_z

12

As you can see, we just added two numbers together while they were still encrypted!!!

One small tweak - <font color=red>notice that since all our numbers are positive, it's possible for each share to reveal a little bit of information about the hidden value, namely, it's always greater than the share. Thus, if Bob has a share "3" then he knows that the encrypted value is at least 3.</font>

This would be quite bad, but can be solved through a simple fix. <font color=red>Decryption happens by summing all the shares together MODULUS some constant. I.e.</font>

In [30]:
x = 5

Q = 23740629843760239486723

bob_x_share = 23552870267 # <- a random number
alice_x_share = Q - bob_x_share + x
alice_x_share

23740629843736686616461

In [31]:
(bob_x_share + alice_x_share) % Q

5

So now, as you can see, both shares are wildly larger than the number being shared, meaning that individual shares no longer leak this inforation. However, all the properties we discussed earlier still hold! (addition, encryption, decryption, etc.)

# Project: Build Methods for Encrypt, Decrypt, and Add 

In this project, you must take the lessons we learned in the last section and write general methods for encrypt, decrypt, and add. Store shares for a variable in a tuple like so.

In [32]:
x_share = (2,5,7)

Even though normally those shares would be distributed amongst several workers, you can store them in ordered tuples like this for now :)

In [33]:
# try this project here!

In [34]:
import random

In [35]:
Q = 231515164481911616167

In [36]:
x = 5

In [37]:
# def encrypt(x, n_shares=3):
n_shares=3
shares = list()

for i in range(n_shares - 1):
    shares.append(random.randint(0,Q))

In [38]:
final_share = Q - (sum(shares) % Q) + x

In [39]:
shares.append(final_share)

In [40]:
sum(shares) % Q

5

In [41]:
def encrypt(x, n_shares=3):
    
    shares = list()

    for i in range(n_shares - 1):
        shares.append(random.randint(0,Q))
    
    final_share = Q - (sum(shares) % Q) + x
    
    shares.append(final_share)
    
    return tuple(shares)

In [42]:
encrypt(5, n_shares=10)

(113487348445637381127,
 202769025327670326794,
 103838498272519032335,
 6086509327767136741,
 125926305511677396284,
 215838477824004097871,
 131529303584993529043,
 62869706324339573058,
 78985698501844766124,
 116244949289104841463)

In [43]:
def decrypt(shares):
    return sum(shares) % Q

In [44]:
def add(a, b):
    
    c = list()
    
    assert(len(a) == len(b))
    
    for i in range(len(a)):
        c.append((a[i] + b[i]) % Q)
        
    return tuple(c)

In [45]:
decrypt(add(encrypt(5), encrypt(10)))

15

In [46]:
x = encrypt(5)
x

(113515846896837897947, 42705934362361490893, 75293383222712227332)

In [47]:
y = encrypt(10)
y

(121798402706250638646, 34635504950014820849, 75081256825646156682)

In [48]:
z = add(x, y)
z

(3799085121176920426, 77341439312376311742, 150374640048358384014)

In [49]:
decrypt(z)

15

# Lesson: Intro to Fixed Precision Encoding

As you may remember, our goal is to aggregate gradients using this new Secret Sharing technique. However, the protocol we've just explored in the last section uses positive integers. However, our neural network weights are NOT integers. Instead, our weights are decimals (floating point numbers).

Not a huge deal! We just need to <font color=red>use a fixed precision encoding, which lets us do computation over decimal numbers using integers!</font>

In [50]:
BASE = 10
PRECISION = 4
Q = 231515164481911616167

In [51]:
def encode(x):
    return int((x * (BASE ** PRECISION)) % Q)

def decode(x):
    return (x if x <= Q/2 else x - Q) / BASE**PRECISION

In [54]:
encode(Q/2 + 0.5)

49545216

In [55]:
x = Q/2 + 0.5
x if x <= Q/2 else x - Q

1.1575758224095581e+20

In [56]:
decode(5000 + 5000)

1.0

In [57]:
x = encrypt(encode(5.5))
y = encrypt(encode(2.3))
z = add(x,y)
decode(decrypt(z))

7.8

In [58]:
x, y

((215601576630652182254, 10139495501242277595, 5774092350017211318),
 (199589978027026595511, 4519721271484599412, 27405465183400444244))

# Lesson: Secret Sharing + Fixed Precision in PySyft

While writing things from scratch is certainly educational, PySyft makes a great deal of this much easier for us through its abstractions.

In [59]:
bob = bob.clear_objects()
alice = alice.clear_objects()
secure_worker = secure_worker.clear_objects()

In [60]:
x = th.tensor([1,2,3,4,5])

### Secret Sharing Using PySyft

We can share using the simple .share() method!

In [61]:
x = x.share(bob, alice, secure_worker)
x

(Wrapper)>[AdditiveSharingTensor]
	-> (Wrapper)>[PointerTensor | me:31383231321 -> bob:78606724036]
	-> (Wrapper)>[PointerTensor | me:21633862981 -> alice:68026945085]
	-> (Wrapper)>[PointerTensor | me:1629538119 -> secure_worker:49975432901]
	*crypto provider: me*

In [62]:
bob._objects

{78606724036: tensor([ 378704029231295553, 4257548163688683249, 3423168811074380024,
         4587209947302896514, 1705005255911385785])}

and as you can see, Bob now has one of the shares of x! Furthermore, we can still call addition in this state, and PySyft will automatically perform the remote execution for us!

In [63]:
y = x + x

In [64]:
bob._objects

{78606724036: tensor([ 378704029231295553, 4257548163688683249, 3423168811074380024,
         4587209947302896514, 1705005255911385785]),
 84614506266: tensor([ 757408058462591106, 8515096327377366498, 6846337622148760048,
         9174419894605793028, 3410010511822771570])}

In [65]:
y

(Wrapper)>[AdditiveSharingTensor]
	-> (Wrapper)>[PointerTensor | me:78399565633 -> bob:84614506266]
	-> (Wrapper)>[PointerTensor | me:6176563059 -> alice:71752099436]
	-> (Wrapper)>[PointerTensor | me:71597598001 -> secure_worker:52410506470]
	*crypto provider: me*

In [66]:
y.get()

tensor([ 2,  4,  6,  8, 10])

In [67]:
bob._objects

{78606724036: tensor([ 378704029231295553, 4257548163688683249, 3423168811074380024,
         4587209947302896514, 1705005255911385785])}

### Fixed Precision using PySyft

We can also convert a tensor to fixed precision using .fix_precision()

In [68]:
x = th.tensor([0.1,0.2,0.3])

In [69]:
x

tensor([0.1000, 0.2000, 0.3000])

In [70]:
x = x.fix_prec()

In [71]:
x

(Wrapper)>FixedPrecisionTensor>tensor([100, 200, 300])

In [72]:
x = x.float_prec()
x

tensor([0.1000, 0.2000, 0.3000])

In [73]:
x = th.tensor([0.1,0.2,0.3])
x

tensor([0.1000, 0.2000, 0.3000])

In [74]:
x = x.fix_prec()
x

(Wrapper)>FixedPrecisionTensor>tensor([100, 200, 300])

In [75]:
type(x)

syft.frameworks.torch.tensors.interpreters.native.Tensor

In [76]:
type(x.child)

syft.frameworks.torch.tensors.interpreters.precision.FixedPrecisionTensor

In [77]:
type(x.child.child)

syft.frameworks.torch.tensors.interpreters.native.Tensor

In [78]:
x.child.child

tensor([100, 200, 300])

In [79]:
y = x + x
y

(Wrapper)>FixedPrecisionTensor>tensor([200, 400, 600])

In [80]:
y = y.float_prec()
y

tensor([0.2000, 0.4000, 0.6000])

### Shared Fixed Precision

And of course, we can combine the two!

In [81]:
x = th.tensor([0.1, 0.2, 0.3])

In [82]:
x = x.fix_prec().share(bob, alice, secure_worker)
x

(Wrapper)>FixedPrecisionTensor>(Wrapper)>[AdditiveSharingTensor]
	-> (Wrapper)>[PointerTensor | me:17474273652 -> bob:82799419356]
	-> (Wrapper)>[PointerTensor | me:48807279366 -> alice:62131787475]
	-> (Wrapper)>[PointerTensor | me:97274875847 -> secure_worker:46434839281]
	*crypto provider: me*

In [83]:
y = x + x
y

(Wrapper)>FixedPrecisionTensor>(Wrapper)>[AdditiveSharingTensor]
	-> (Wrapper)>[PointerTensor | me:49054587695 -> bob:80023346197]
	-> (Wrapper)>[PointerTensor | me:12642202541 -> alice:45154367702]
	-> (Wrapper)>[PointerTensor | me:93565558259 -> secure_worker:28313515335]
	*crypto provider: me*

In [84]:
bob._objects

{78606724036: tensor([ 378704029231295553, 4257548163688683249, 3423168811074380024,
         4587209947302896514, 1705005255911385785]),
 82799419356: tensor([1017458394297488632,  538494362188868660, 1791795266099960875]),
 80023346197: tensor([2034916788594977264, 1076988724377737320, 3583590532199921750])}

In [85]:
y.get().float_prec()

tensor([0.2000, 0.4000, 0.6000])

In [86]:
bob._objects

{78606724036: tensor([ 378704029231295553, 4257548163688683249, 3423168811074380024,
         4587209947302896514, 1705005255911385785]),
 82799419356: tensor([1017458394297488632,  538494362188868660, 1791795266099960875])}

Make sure to make the point that people can see the model averages in the clear.

# Final Project: Federated Learning with Encrypted Gradient Aggregation